# Modèles multimodaux - CLIP

## README
Ce notebook permet la création et l'évaluation d'un modèle basé sur l'architecture existatne CLIP.

## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import os
import time
import pandas as pd

from src.models.multimodal_clip import MultiModalCLIPBasedClassifier
from src.models.model_wrappers import ModelWrapperFactory
from src.visualization.visualize import plot_history

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam

## 2. Chargement des données
On charge directement les embeddings sortant de CLIP

In [ ]:
documents = pd.read_parquet(PATHS.metadata / "df_data_sets.parquet")
data_sets = pd.read_parquet(PATHS.metadata / "df_data_sets.parquet")
labels = pd.read_parquet(PATHS.metadata / "df_encoded_labels.parquet")

In [ ]:
# pour ne travailler que sur un échantillon
sample = pd.read_parquet(os.path.join(PATHS.metadata, 'samples', 'df_documents_sample_4k_2.parquet'))
documents = sample.join(documents)
labels = sample.join(labels)
data_sets = sample.join(data_sets)

In [ ]:
documents.shape, data_sets.shape, labels.shape

In [ ]:
X_train = documents[data_sets.data_set == "train"].index
y_train = labels[data_sets.data_set == "train"].label

X_val = documents[data_sets.data_set == "val"].index
y_val = labels[data_sets.data_set == "val"].label

X_test = documents[data_sets.data_set == "test"].index
y_test = labels[data_sets.data_set == "test"].label

In [ ]:
del documents, data_sets, labels

## 3. Création des callbacks

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

def reduce_lr():
    return ReduceLROnPlateau(
    monitor='val_loss',     
    factor=0.5,             
    patience=3,             
    min_lr=1e-6,            
    verbose=1              
)

def checkpoint():
    return ModelCheckpoint(
    filepath=os.path.join(PATHS.models, 'checkpoints', 'best_model.keras'),
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)
os.makedirs(os.path.join(PATHS.models, 'checkpoints'), exist_ok=True)
            
def early_stop():
    return EarlyStopping(
    monitor='val_loss',       
    patience=10,
    restore_best_weights=True,
    verbose=1
)

## 4. Modèles

### 4.1. Modèle 1

In [ ]:
# Création du modèle
clf = Sequential([
    Input((1024,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
clf.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop(), checkpoint(), reduce_lr()],
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp1.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model1"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.2. Modèle 2

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Input((1024,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp2.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model2"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.3. Modèle 3

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-5)
clf = Sequential([
    Input((1024,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp3.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model3"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.4. Modèle 4

In [ ]:
# Création du modèle
clf = Sequential([
    Input((1024,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
clf.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp4.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model4"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.5. Modèle 5

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Input((1024,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp5.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model5"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.6. Modèle 6

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Input((1024,)),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp6.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model6"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.7. Modèle 7

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Input((1024,)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(16, activation='softmax')
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp7.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model7"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.8. Modèle 8

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Input((1024,)),
    Dense(256),
    LeakyReLU(alpha=0.1),
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp8.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model8"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.9. Modèle 9

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Input((1024,)),
    Dense(256),
    Dropout(0.4),
    LeakyReLU(alpha=0.1),
    Dense(128),
    Dropout(0.2),
    LeakyReLU(alpha=0.1),
    Dense(128),
    Dropout(0.2),
    LeakyReLU(alpha=0.1),
    Dense(64),
    Dropout(0.1),
    LeakyReLU(alpha=0.1),
    Dense(64),
    Dropout(0,1),
    LeakyReLU(alpha=0.1),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp9.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model9"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.10. Modèle 10

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Input((1024,)),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp10.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model10"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.11. Modèle 11

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Input((1024,)),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp11.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model11"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()

### 4.12. Modèle 12

In [ ]:
# Création du modèle
optimizer = Adam(learning_rate=1e-4)
clf = Sequential([
    Dense(512),
    LeakyReLU(alpha=0.1),
    Dropout(0.4),
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(16, activation='softmax')  # 16 classes
])

# Compilation
clf.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model = MultiModalCLIPBasedClassifier(classifier=clf)

# Entrainement
t0 = time.time()
train_history = model.fit(
                    X_train,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr(), early_stop(), checkpoint()]
                )
print(f"Entraînement réalisé en {time.time()-t0:.2f} secondes.")

# sauvegarde du modele et affichage des résultats
path = PATHS.models / "mmo_clip+mlp12.keras"
model.save(path)

plot_history(train_history)
name = "CLIP-based NN Model12"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()